In [12]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from playlist import optimized_pomodoro_playlist  # Import your logic

# List all possible patterns, mapping string to integer class
PATTERN_LIST = [
    "WSW",
    "WSWSWL",
    "WSWSWSWL",
    "WSWSWSWL+WSWS",
    "2×WSWSWL",
    "2×WSWSWSWL",
]
PATTERN_TO_IDX = {p: i for i, p in enumerate(PATTERN_LIST)}
IDX_TO_PATTERN = {i: p for p, i in PATTERN_TO_IDX.items()}
MAX_SESSIONS = 8  # Maximum number of work sessions across all patterns

def generate_dataset():
    X, y_pattern, y_sessions, y_breaks = [], [], [], []
    for mins in range(30, 360 + 1, 5):
        result = optimized_pomodoro_playlist(f"{mins}:00", code_format=False)
        # Input: duration as float
        X.append([float(mins)])
        # Pattern class (int)
        y_pattern.append(PATTERN_TO_IDX[result["sequence"]])
        # Work sessions (list, pad to MAX_SESSIONS with 0s)
        ws = result["work_sessions"] + [0]*(MAX_SESSIONS-len(result["work_sessions"]))
        y_sessions.append(ws)
        # Short/Long break
        short = result["short_break"]
        long_ = result["long_break"] if result["long_break"] is not None else 0
        y_breaks.append([short, long_])
    return np.array(X), np.array(y_pattern), np.array(y_sessions), np.array(y_breaks)

X, y_pattern, y_sessions, y_breaks = generate_dataset()


In [13]:
class PomodoroDataset(Dataset):
    def __init__(self, X, y_pattern, y_sessions, y_breaks):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_pattern = torch.tensor(y_pattern, dtype=torch.long)
        self.y_sessions = torch.tensor(y_sessions, dtype=torch.float32)
        self.y_breaks = torch.tensor(y_breaks, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y_pattern[idx], self.y_sessions[idx], self.y_breaks[idx]

dataset = PomodoroDataset(X, y_pattern, y_sessions, y_breaks)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [14]:
import torch.nn as nn

class PomodoroNet(nn.Module):
    def __init__(self, num_patterns, max_sessions):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.pattern_head = nn.Linear(64, num_patterns)      # Classification
        self.sessions_head = nn.Linear(64, max_sessions)     # Regression
        self.breaks_head = nn.Linear(64, 2)                  # Regression

    def forward(self, x):
        features = self.shared(x)
        pattern_logits = self.pattern_head(features)
        sessions_pred = self.sessions_head(features)
        breaks_pred = self.breaks_head(features)
        return pattern_logits, sessions_pred, breaks_pred


In [19]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PomodoroNet(num_patterns=len(PATTERN_LIST), max_sessions=MAX_SESSIONS).to(device)
criterion_class = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 4000

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x, y_pat, y_sess, y_break in train_loader:
        x, y_pat, y_sess, y_break = x.to(device), y_pat.to(device), y_sess.to(device), y_break.to(device)
        optimizer.zero_grad()
        pat_logits, sess_pred, break_pred = model(x)
        loss_pat = criterion_class(pat_logits, y_pat)
        loss_sess = criterion_reg(sess_pred, y_sess)
        loss_break = criterion_reg(break_pred, y_break)
        loss = loss_pat + loss_sess + loss_break
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch+1) % 50 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/4000 - Loss: 1388.1778
Epoch 50/4000 - Loss: 106.6095
Epoch 100/4000 - Loss: 113.0415
Epoch 150/4000 - Loss: 109.9143
Epoch 200/4000 - Loss: 109.2724
Epoch 250/4000 - Loss: 113.2789
Epoch 300/4000 - Loss: 98.7195
Epoch 350/4000 - Loss: 105.6547
Epoch 400/4000 - Loss: 99.4540
Epoch 450/4000 - Loss: 116.2709
Epoch 500/4000 - Loss: 99.1657
Epoch 550/4000 - Loss: 95.5776
Epoch 600/4000 - Loss: 104.2946
Epoch 650/4000 - Loss: 99.3476
Epoch 700/4000 - Loss: 98.2269
Epoch 750/4000 - Loss: 88.1567
Epoch 800/4000 - Loss: 79.1940
Epoch 850/4000 - Loss: 84.1394
Epoch 900/4000 - Loss: 66.3664
Epoch 950/4000 - Loss: 68.1198
Epoch 1000/4000 - Loss: 56.1194
Epoch 1050/4000 - Loss: 68.1148
Epoch 1100/4000 - Loss: 62.2308
Epoch 1150/4000 - Loss: 52.1847
Epoch 1200/4000 - Loss: 61.7455
Epoch 1250/4000 - Loss: 50.7458
Epoch 1300/4000 - Loss: 53.6873
Epoch 1350/4000 - Loss: 56.5480
Epoch 1400/4000 - Loss: 47.5772
Epoch 1450/4000 - Loss: 64.0207
Epoch 1500/4000 - Loss: 64.3245
Epoch 1550/4000 - Los

In [20]:
def predict(duration_minutes: float) -> dict:
    model.eval()
    x = torch.tensor([[duration_minutes]], dtype=torch.float32).to(device)
    with torch.no_grad():
        pat_logits, sess_pred, break_pred = model(x)
        pattern_idx = torch.argmax(pat_logits, dim=1).item()
        pattern = IDX_TO_PATTERN[pattern_idx]

        sessions = sess_pred[0].cpu().round().clamp(min=0).int().tolist()
        needed = pattern.count('W')
        sessions = sessions[:needed]
        
        breaks = break_pred[0].cpu().round().clamp(min=0).int().tolist()
        short_break = breaks[0]
        long_break = breaks[1] if 'L' in pattern else None

        # Enforce: long_break > short_break if long_break exists
        if long_break is not None:
            if long_break <= short_break:
                # Option 1: Make long_break at least short_break + 5
                long_break = short_break + 5

    return {
        "duration_minutes": duration_minutes,
        "pattern": pattern,
        "work_sessions": sessions,
        "short_break": short_break,
        "long_break": long_break
    }
# Example usage:
print(predict(115))


{'duration_minutes': 115, 'pattern': 'WSWSWL', 'work_sessions': [30, 30, 28], 'short_break': 5, 'long_break': 12}


In [21]:
# Evaluate pattern prediction accuracy and MAE for regression outputs
def evaluate(model, dataset):
    model.eval()
    correct = 0
    total = 0
    session_mae = 0
    break_mae = 0
    with torch.no_grad():
        for x, y_pat, y_sess, y_break in DataLoader(dataset, batch_size=64):
            x = x.to(device)
            pat_logits, sess_pred, break_pred = model(x)
            pred_pat = torch.argmax(pat_logits, dim=1)
            correct += (pred_pat.cpu() == y_pat).sum().item()
            total += y_pat.size(0)
            session_mae += (sess_pred.cpu() - y_sess).abs().sum().item()
            break_mae += (break_pred.cpu() - y_break).abs().sum().item()
    n = len(dataset)
    print(f"Pattern accuracy: {correct/total:.2%}")
    print(f"Session MAE: {session_mae/(n*MAX_SESSIONS):.2f}")
    print(f"Break MAE: {break_mae/(n*2):.2f}")

evaluate(model, dataset)


Pattern accuracy: 91.04%
Session MAE: 3.46
Break MAE: 1.17


In [22]:
result_code_str = optimized_pomodoro_playlist("115:00", code_format=True)
print("Coded Output:")
print(result_code_str)
print("\n------------------------------------------------\n")

Coded Output:
115:00:WSWSWL:35:5:30:5:30:10:loss=00:00

------------------------------------------------



In [24]:
torch.save(model.state_dict(), "pomodoro_model.pth")

In [26]:
model = PomodoroNet(num_patterns=len(PATTERN_LIST), max_sessions=MAX_SESSIONS)
model.load_state_dict(torch.load("pomodoro_model.pth"))
model.to(device)  # Move to CUDA/CPU as appropriate
model.eval()      # Set to evaluation mode (important for inference)

predict(115)

{'duration_minutes': 115,
 'pattern': 'WSWSWL',
 'work_sessions': [30, 30, 28],
 'short_break': 5,
 'long_break': 12}